In [60]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense
from IPython.display import display, HTML
import matplotlib.pyplot as plt


In [59]:
data = pd.read_csv("archive (3)/train.txt", sep=';')
data.columns = ["Text", "Emotions"]
print(data.head())

                                                Text Emotions
0  i can go from feeling so hopeless to so damned...  sadness
1   im grabbing a minute to post i feel greedy wrong    anger
2  i am ever feeling nostalgic about the fireplac...     love
3                               i am feeling grouchy    anger
4  ive been feeling a little burdened lately wasn...  sadness


In [61]:
# Preprocess the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Text'])

# Convert texts to sequences of integers
X = tokenizer.texts_to_sequences(data['Text'])

# Padding sequences to make them of equal length
max_length = max([len(seq) for seq in X])  # Calculate maximum sequence length
X_pad = pad_sequences(X, maxlen=max_length, padding='post')

# Encoding labels (emotions)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Emotions'])


In [65]:
# Extract the text and emotion labels
texts = data["Text"].tolist()
labels = data["Emotions"].tolist()

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)

# Convert texts to sequences
sequences = tokenizer.texts_to_sequences(texts)

# Find the maximum sequence length
max_length = max([len(seq) for seq in sequences])

# Pad the sequences to make them of uniform length
padded_sequences = pad_sequences(sequences, maxlen=max_length)

# Display the first few padded sequences for verification
print("Sample Padded Sequences:")
print(padded_sequences[:5])


Sample Padded Sequences:
[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    1   39  100   59    7   14  493    4   14 3495  552
    31   59   60  127  147   75 1479    3   21 1254]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
    16 3059    6 1148    4  285    1    2  494  437]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    1   23  164    7  664   26    5 4157
     

In [66]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# Encode the string emotion labels to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# One-hot encode the integer labels
one_hot_labels = to_categorical(labels)

# Split the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(padded_sequences, 
                                                one_hot_labels, 
                                                test_size=0.2, 
                                                random_state=42)

# Display a sample one-hot encoded label and the shapes of train/test sets
print("Sample One-Hot Encoded Label:", one_hot_labels[0])
print("\nShape of Training Data (xtrain):", xtrain.shape)
print("Shape of Testing Data (xtest):", xtest.shape)
print("Shape of Training Labels (ytrain):", ytrain.shape)
print("Shape of Testing Labels (ytest):", ytest.shape)


Sample One-Hot Encoded Label: [0. 0. 0. 0. 1. 0.]

Shape of Training Data (xtrain): (12799, 66)
Shape of Testing Data (xtest): (3200, 66)
Shape of Training Labels (ytrain): (12799, 6)
Shape of Testing Labels (ytest): (3200, 6)


In [68]:
# Import necessary libraries
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Flatten, Dense

# Build the model
model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=128),  # Removed `input_length`
    Flatten(),
    Dense(units=128, activation='relu'),
    Dense(units=len(label_encoder.classes_), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(xtrain, ytrain, epochs=5, batch_size=32, validation_data=(xtest, ytest))


Epoch 1/5
400/400 ━━━━━━━━━━━━━━━━━━━━ 18s 37ms/step - accuracy: 0.3933 - loss: 1.5218 - val_accuracy: 0.6969 - val_loss: 0.8762
Epoch 2/5
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 36ms/step - accuracy: 0.8729 - loss: 0.4148 - val_accuracy: 0.8194 - val_loss: 0.5485
Epoch 3/5
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 34ms/step - accuracy: 0.9856 - loss: 0.0593 - val_accuracy: 0.8222 - val_loss: 0.5753
Epoch 4/5
400/400 ━━━━━━━━━━━━━━━━━━━━ 14s 34ms/step - accuracy: 0.9962 - loss: 0.0209 - val_accuracy: 0.8181 - val_loss: 0.6081
Epoch 5/5
400/400 ━━━━━━━━━━━━━━━━━━━━ 13s 33ms/step - accuracy: 0.9970 - loss: 0.0154 - val_accuracy: 0.8150 - val_loss: 0.6678


In [93]:
# Importing required libraries
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from sklearn.preprocessing import LabelEncoder
import keras
from IPython.display import display, HTML, Javascript
import random
import time

# Define the dictionary for 50 emotions to emojis
emotion_to_emoji = {
    "sadness": "😢", "anger": "😡", "love": "❤️", "surprise": "😮", "fear": "😱",
    "happiness": "😊", "joy": "😂", "disgust": "🤢", "boredom": "😒", "pride": "😎",
    "gratitude": "🙏", "embarrassment": "😳", "confusion": "😕", "hope": "🤞", "relief": "😌",
    "shame": "😳", "anxiety": "😰", "excitement": "🤩", "frustration": "😤", "love-sick": "💔",
    "curiosity": "🤔", "peace": "☮️", "nostalgia": "😌", "contempt": "😒", "guilt": "😔",
    "priceless": "🤩", "determination": "💪", "serenity": "🌿", "inspiration": "💡", "enthusiasm": "🎉",
    "confident": "💃", "affection": "🥰", "sympathy": "🤗", "mischief": "😈", "longing": "💭",
    "hopeful": "🌈", "embarrassed": "🙈", "shocked": "😲", "flirt": "😘", "dreamy": "🌙",
    "exhilarated": "😝", "optimistic": "☀️", "pessimistic": "🌧️", "awestruck": "🤯", "contentment": "😌",
    "affectionate": "❤️", "humor": "🤣", "grateful": "🙌", "relaxed": "😌", "sleepy": "😴",
    "peaceful": "🕊️", "energetic": "⚡"
}

# Define a function to predict the emotion with emoji
def predict_emotion_with_emoji(text):
    # Preprocess the input text
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)

    # Predict the emotion
    prediction = model.predict(padded_sequence)
    predicted_emotion = label_encoder.inverse_transform([prediction.argmax()])[0]
    
    # Get the emoji for the predicted emotion or use a default one
    emoji = emotion_to_emoji.get(predicted_emotion, "❓")
    
    return predicted_emotion, emoji

# Function to display the output beautifully with a moving emoji and emotion
def display_result(input_text):
    predicted_emotion, emoji = predict_emotion_with_emoji(input_text)

    # HTML format for animated emoji display
    html_output = f"""
    <div style="
        font-family: Arial, sans-serif;
        padding: 20px;
        border-radius: 10px;
        max-width: 400px;
        margin: auto;
        background: url('https://wallpapercave.com/wp/wp2832043.jpg');
        background-size: cover;
        box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
        border: 5px solid white;
    ">
        <h2 style="font-size: 24px; color: white; text-shadow: 2px 2px 4px black; text-align: center;">Emotion Detection</h2>
        <p style="font-size: 18px; text-align: center; color: white; text-shadow: 1px 1px 2px black;"><strong>Input Text:</strong> {input_text}</p>
        <p style="font-size: 80px; text-align: center; animation: moveEmoji 2s infinite;">{emoji}</p>
        <p style="font-size: 20px; text-align: center; color: white; text-shadow: 1px 1px 2px black;"><strong>Predicted Emotion:</strong> {predicted_emotion}</p>
    </div>
    <style>
        @keyframes moveEmoji {{
            0% {{ transform: translateY(0); }}
            50% {{ transform: translateY(-10px); }}
            100% {{ transform: translateY(0); }}
        }}
    </style>
    """
    display(HTML(html_output))

# Input text
input_text = input("Enter a sentence to detect emotion: ")  # You can enter the sentence here
display_result(input_text)


Enter a sentence to detect emotion:  I miss the good old days when everything felt easier.


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 622ms/step
